In [22]:
f = GF(7)
n = 10

In [23]:
sparse = Matrix(f, [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 3, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 5, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 4, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 0, 0],
    [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
])

In [24]:
toeplitz = Matrix(f, [
    [1, 2, 3, 4, 5, 6, 0, 1, 2, 3],
    [0, 1, 2, 3, 4, 5, 6, 0, 1, 2],
    [6, 0, 1, 2, 3, 4, 5, 6, 0, 1],
    [5, 6, 0, 1, 2, 3, 4, 5, 6, 0],
    [4, 5, 6, 0, 1, 2, 3, 4, 5, 6],
    [3, 4, 5, 6, 0, 1, 2, 3, 4, 5],
    [2, 3, 4, 5, 6, 0, 1, 2, 3, 4],
    [1, 2, 3, 4, 5, 6, 0, 1, 2, 3],
    [0, 1, 2, 3, 4, 5, 6, 0, 1, 2],
    [6, 0, 1, 2, 3, 4, 5, 6, 0, 1],
])

In [25]:
hankel = [[(i + j) % 7 for j in range(10)] for i in range(10)]
# adjust so that anti-diagonals are constant
hankel = [[(i + j) % 7 for j in range(10)] for i in range(10)]
for i in range(10):
    for j in range(10):
        hankel[i][j] = (i + j) % 7
hankel = Matrix(f, hankel)

In [26]:
diff_diagonal = [[0]*10 for _ in range(10)]
for i in range(10):
    diff_diagonal[i][i] = (i + 1) % 7
diff_diagonal = Matrix(f, diff_diagonal)

In [27]:
same_diagonal = [[0]*10 for _ in range(10)]
for i in range(10):
    same_diagonal[i][i] = 2
same_diagonal = Matrix(f, same_diagonal)

In [28]:
sylvester = Matrix(f, [
    [1, 0, 2, 0, 0, 6, 0, 0, 0, 0],
    [0, 1, 0, 2, 0, 0, 6, 0, 0, 0],
    [0, 0, 1, 0, 2, 0, 0, 6, 0, 0],
    [0, 0, 0, 1, 0, 2, 0, 0, 6, 0],
    [0, 0, 0, 0, 1, 0, 2, 0, 0, 6],
    [1, 0, 3, 0, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 3, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 3, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 3, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 3, 0, 0, 1],
])

In [29]:
jordan_block = [[0]*10 for _ in range(10)]
λ = 3
for i in range(10):
    jordan_block[i][i] = λ
    if i < 9:
        jordan_block[i][i+1] = 1
jordan_block = Matrix(f, jordan_block)

In [30]:
companion = [[0]*10 for _ in range(10)]
for i in range(9):
    companion[i+1][i] = 1
coeffs = [2, 5, 4, 3, 6, 1, 0, 2, 1, 6]  # a_0 to a_9
for j in range(10):
    companion[0][j] = (-coeffs[9-j]) % 7
companion = Matrix(f, companion)

In [31]:
v = [i % 7 for i in range(10)]
vandermonde = Matrix(f, [[pow(v[j], i, 7) for j in range(10)] for i in range(10)])


In [32]:
from sage.matrix.berlekamp_massey import berlekamp_massey
import random

def random_vector(n, f):
    return vector(f, [f.random_element() for _ in range(n)])

In [33]:
mats = [sparse, toeplitz, hankel, diff_diagonal, same_diagonal, sylvester, jordan_block, companion, vandermonde]
def avg_early_term(A, trials = 1000):
    # write a function that runs 1000 times see on average how many krylov terms 
    # is needed to recover actual min poly
    total = 0
    minpoly = A.minpoly()
    mu = 0
    for _ in range(trials):
        k = 0
        u = random_vector(n, f)
        v = random_vector(n, f)
        while k < n:
            k += 1
            kappa = [u.dot_product((A^i)*v) for i in range(2*k)]
            mu = berlekamp_massey(kappa)
            mu = mu / mu.leading_coefficient()
            if mu == minpoly: 
                break
        total += 2*k
        if 2*k != 2*n:
            print(2*k)
    return float(total/trials)

In [34]:
# for mat in mats: 
#     avg_early_term(mat)
#     print('---')

In [35]:
# def print_early_term(A):
#         minpoly = A.minpoly()
#         mu = 0
#         k = 1
#         u = random_vector(n, f)
#         v = random_vector(n, f)
#         while k <= n: 
#             kappa = [u.dot_product((A^i)*v) for i in range(2*k)]
#             mu = berlekamp_massey(kappa)
#             mu = mu / mu.leading_coefficient()
#             print(mu)
#             if mu==minpoly:
#                 print('Yay')
#             k += 1
# for mat in mats: 
#     print(f'---Min poly: {mat.minpoly()}---')
#     print_early_term(mat)

In [36]:
def empirical_success(A, p=7, trials=1000):
    f = GF(p)
    n = A.nrows()
    succeeded = 0
    min_poly = A.minpoly()
    
    for _ in range(trials):
        u = random_vector(n, f)
        v = random_vector(n, f)
        kappa = [u.dot_product((A^i)*v) for i in range(2*n)]
        mu = berlekamp_massey(kappa)
        if mu == min_poly:
            succeeded += 1
    prob = succeeded / trials
    return float(prob)

for mat in mats: 
    print(empirical_success(mat))

0.0
0.0
0.0
0.0
0.851
0.671
0.735
0.677
0.0
